In [1]:
#jk
# # for pytorch basics: refer sunny's repo:
# https://github.com/c17hawke/Pytorch-basics

In [3]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt


In [4]:
IMAGE_SIZE = (224, 224)


In [5]:
# just downloading the vgg model to see the architecture
model = tf.keras.applications.VGG16(
    include_top=True,  # whether we want to include the Fully connected layer or not.. If it is 'False', it will give till convolution layer
    weights="imagenet", # already trained on "imagenet" dataset
    input_tensor=None,
    input_shape=None,
    classes=1000
)

553476096/553467096 [==============================] - 113s 0us/step


In [6]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
# save the model
model.save("VGG16_full_model.h5")

In [8]:
model.layers[1].padding

'same'

In [9]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# create a directory


def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")


ROOT_DATA_DIR = "data"
create_dirs(ROOT_DATA_DIR)


data_zip_file = "data.zip"
data_zip_path = os.path.join(ROOT_DATA_DIR, data_zip_file)

if not os.path.isfile(data_zip_file):
    print("downloading data...")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename: {filename} created with info \n{headers}")
else:
    print(f"file is already present")


data directory created
downloading data...
filename: data\data.zip created with info 
Content-Type: application/zip
Content-Length: 47286322
Connection: close
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
Date: Thu, 05 May 2022 06:51:38 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
X-Cache: RefreshHit from cloudfront
Via: 1.1 8c294c60dd2a72491038e243bb35cd76.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: MAA51-P3
X-Amz-Cf-Id: TtvY1ORpXZtbGSHkI9HgdhkbfOw6QXE2VE6F2eRCkmYea33Srgd-ZQ==




In [10]:
from zipfile import ZipFile

unzip_data_dirname = "unzip_data_dir"
unzip_data_dir = os.path.join(ROOT_DATA_DIR, unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir, exist_ok=True)
    with ZipFile(data_zip_path) as f:
        f.extractall(unzip_data_dir)
else:
    print(f"data already extacted")


In [11]:
main_data_dir = Path("./data/unzip_data_dir/hymenoptera_data/train")
main_data_dir


WindowsPath('data/unzip_data_dir/hymenoptera_data/train')

In [12]:
BATCH_SIZE = 32
pixels = 224
IMAGE_SIZE = (pixels, pixels)


In [13]:
datagen_kwargs = dict(rescale=1./255, validation_split=0.20)

dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)


In [14]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)


valid_generator = valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=False, **dataflow_kwargs)


do_data_augmentation = False

if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotational_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        **datagen_kwargs)
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    main_data_dir, subset="training", shuffle=True, **dataflow_kwargs
)


Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [15]:
RGB_IMAGE_SIZE = (pixels, pixels, 3)


vgg = tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False  # just looking for convolution layer
)

58900480/58889256 [==============================] - 11s 0us/step


In [16]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")


input_2             : True
block1_conv1        : True
block1_conv2        : True
block1_pool         : True
block2_conv1        : True
block2_conv2        : True
block2_pool         : True
block3_conv1        : True
block3_conv2        : True
block3_conv3        : True
block3_pool         : True
block4_conv1        : True
block4_conv2        : True
block4_conv3        : True
block4_pool         : True
block5_conv1        : True
block5_conv2        : True
block5_conv3        : True
block5_pool         : True


In [18]:
for layer in vgg.layers:
    layer.trainable = False


In [19]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")


input_2             : False
block1_conv1        : False
block1_conv2        : False
block1_pool         : False
block2_conv1        : False
block2_conv2        : False
block2_pool         : False
block3_conv1        : False
block3_conv2        : False
block3_conv3        : False
block3_pool         : False
block4_conv1        : False
block4_conv2        : False
block4_conv3        : False
block4_pool         : False
block5_conv1        : False
block5_conv2        : False
block5_conv3        : False
block5_pool         : False


In [20]:
vgg.output


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [21]:
del model


In [22]:
CLASSES = 2
x = tf.keras.layers.Flatten()(vgg.output)
prediction = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs=prediction)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)


In [24]:
EPOCHS = 10

history = model.fit(
    train_generator, epochs=EPOCHS, validation_data=valid_generator
)


Epoch 1/10
7/7 [==============================] - 110s 15s/step - loss: 15.1014 - accuracy: 0.4439 - val_loss: 29.7442 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 109s 16s/step - loss: 22.3154 - accuracy: 0.5663 - val_loss: 1.0808 - val_accuracy: 0.7917
Epoch 3/10
7/7 [==============================] - 117s 17s/step - loss: 17.1745 - accuracy: 0.5714 - val_loss: 26.7158 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 110s 18s/step - loss: 19.7799 - accuracy: 0.5867 - val_loss: 7.6443 - val_accuracy: 0.7500
Epoch 5/10
7/7 [==============================] - 121s 18s/step - loss: 11.0110 - accuracy: 0.7296 - val_loss: 5.7866 - val_accuracy: 0.7917
Epoch 6/10
7/7 [==============================] - 118s 19s/step - loss: 11.9303 - accuracy: 0.7296 - val_loss: 14.1295 - val_accuracy: 0.6458
Epoch 7/10
7/7 [==============================] - 120s 17s/step - loss: 7.5721 - accuracy: 0.7551 - val_loss: 18.9591 - val_accuracy: 0.6042
Epoch 8/10